# Rundll32 Internet Connection
Detects a rundll32 that communicates with public IP addresses

## Rule Content
```
- title: Rundll32 Internet Connection
  id: cdc8da7d-c303-42f8-b08c-b4ab47230263
  status: experimental
  description: Detects a rundll32 that communicates with public IP addresses
  references:
  - https://www.hybrid-analysis.com/sample/759fb4c0091a78c5ee035715afe3084686a8493f39014aea72dae36869de9ff6?environmentId=100
  author: Florian Roth
  date: 2017/11/04
  tags:
  - attack.t1085
  - attack.defense_evasion
  - attack.execution
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 3
      Image: '*\rundll32.exe'
      Initiated: 'true'
    filter:
      DestinationIp:
      - 10.*
      - 192.168.*
      - 172.16.*
      - 172.17.*
      - 172.18.*
      - 172.19.*
      - 172.20.*
      - 172.21.*
      - 172.22.*
      - 172.23.*
      - 172.24.*
      - 172.25.*
      - 172.26.*
      - 172.27.*
      - 172.28.*
      - 172.29.*
      - 172.30.*
      - 172.31.*
      - 127.*
    condition: selection and not filter
  falsepositives:
  - Communication to other corporate systems that use IP addresses from public address
    spaces
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"3" AND process_path.keyword:*\\rundll32.exe AND network_initiated:"true") AND (NOT (dst_ip_addr.keyword:(10.* OR 192.168.* OR 172.16.* OR 172.17.* OR 172.18.* OR 172.19.* OR 172.20.* OR 172.21.* OR 172.22.* OR 172.23.* OR 172.24.* OR 172.25.* OR 172.26.* OR 172.27.* OR 172.28.* OR 172.29.* OR 172.30.* OR 172.31.* OR 127.*))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()